In [0]:
!pip3 -qq install torch==0.4.1
!pip install -qq bokeh==0.13.0
!pip install -qq eli5==0.8
!wget -O surnames.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1z7avv1JiI30V4cmHJGFIfDEs9iE4SHs5"

In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
else:
    from torch import FloatTensor, LongTensor
    
np.random.seed(42)

# Свёрточные нейронные сети

## Классификация фамилий

Будем учиться предсказывать, является ли слово фамилией.

In [0]:
from sklearn.model_selection import train_test_split

with open('surnames.txt') as f:
    lines = f.readlines()
    data = [line.strip().split('\t')[0] for line in lines]
    labels = np.array([int(line.strip().split('\t')[1]) for line in lines])
    del lines
    
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.33, random_state=42)

Посмотрим на данные:

In [0]:
list(zip(train_data, train_labels))[::1500]

Данные ещё и сильно несбалансированы - положительных примеров в несколько раз меньше:

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

positive_count = np.sum(train_labels == 1)
negative_count = len(train_labels) - positive_count
 
plt.bar(np.arange(2), [negative_count, positive_count], align='center', alpha=0.5)
plt.xticks(np.arange(2), ('Negative', 'Positive'))
    
plt.show()

Accuracy очень легко оптимизировать - просто предсказывайте всегда ноль:

In [0]:
print('Accuracy = {:.2%}'.format((train_labels == 0).mean()))

Однако это будет довольно бесполезно - всегда говорить, что слово не является фамилией. Это, конечно, вопрос - что хуже, зря объявить слово фамилией (ошибка первого рода) или не найти фамилию.

![](https://effectsizefaq.files.wordpress.com/2010/05/type-i-and-type-ii-errors.jpg =x350)  
*From [effectsizefaq.com](https://effectsizefaq.com/2010/05/31/i-always-get-confused-about-type-i-and-ii-errors-can-you-show-me-something-to-help-me-remember-the-difference/)*

Будем замерять precision, recall и их комбинацию - $F_1$-меру.

![precision-recall](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/350px-Precisionrecall.svg.png =x600)  
*From [Wikipedia](https://en.wikipedia.org/wiki/Precision_and_recall)*

$$\text{precision} = \frac{tp}{tp + fp}.$$
$$\text{recall} = \frac{tp}{tp + fn}.$$
$$\text{F}_1 = 2 \cdot \frac{\text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}}.$$

Начнём с бейзлайна на регулярках.

In [0]:
#@title Супер-бейзлайн
surname_indicators = "^[А-Я][а-я], .*ский" #@param {type:"raw"}

surname_indicators = surname_indicators.split(', ')

import re

regexs = [re.compile(regex) for regex in surname_indicators]

preds = np.array([any(regex.match(word) for regex in regexs) for word in test_data])

from sklearn.metrics import f1_score
print('F1-score = {:.2%}'.format(f1_score(test_labels, preds)))

А теперь серьёзно - бейзлайн на логистической регрессии поверх N-грамм символов.

**Задание** Сделать классификацию с LogisticRegression моделью. Посчитать F1-меру.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 3), lowercase=False)

X_train, X_test = <convert data>

model = <fit LogisticRegression>

test_preds = model.predict(X_test)
print('F1-score = {:.2%}'.format(f1_score(test_labels, test_preds)))

Посмотрим на предсказания

In [0]:
import eli5

eli5.show_weights(model, vec=vectorizer, top=40)

In [0]:
sample_ind = np.random.randint(len(test_data))
eli5.show_prediction(model, test_data[sample_ind], vec=vectorizer, targets=['surname'], target_names=['word', 'surname'])

Кроме тупого подсчета F1-score можно посмотреть на precision-recall кривые. Во-первых, они красивые. Во-вторых, по ним видно, что можно повысить качество (F1-score), подобрав другой порог - **хотя на тесте это делать нельзя**.

In [0]:
from sklearn.metrics import precision_recall_curve

precision, recall, _ = precision_recall_curve(test_labels, model.predict_proba(X_test)[:, 1])

plt.figure(figsize=(7, 7))
f_scores = np.linspace(0.2, 0.8, num=4)
lines = []
labels = []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
    plt.annotate('F1 = {0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

plt.plot(recall, precision)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')

plt.show()

**Задание** Придумайте признаки, чтобы улучшить качество модели.

## Character-Level Convolutions

### Общее описание сверток

Напомню, свертки - это то, с чего начался хайп нейронных сетей в районе 2012-ого.

Работают они примерно так:  
![Conv example](https://image.ibb.co/e6t8ZK/Convolution.gif)   
From [Feature extraction using convolution](http://deeplearning.stanford.edu/wiki/index.php/Feature_extraction_using_convolution).

Формально - учатся наборы фильтров, каждый из которых скалярно умножается на элементы матрицы признаков. На картинке выше исходная матрица сворачивается с фильтром
$$
 \begin{pmatrix}
  1 & 0 & 1 \\
  0 & 1 & 0 \\
  1 & 0 & 1
 \end{pmatrix}
$$

Но нужно не забывать, что свертки обычно имеют ещё такую размерность, как число каналов. Например, картинки имеют обычно три канала: RGB.  
Наглядно демонстрируется как выглядят при этом фильтры [здесь](http://cs231n.github.io/convolutional-networks/#conv).

После сверток обычно следуют pooling-слои. Они помогают уменьшить размерность тензора, с которым приходится работать. Самым частым является max-pooling:  
![maxpooling](http://cs231n.github.io/assets/cnn/maxpool.jpeg =x300)  
From [CS231n Convolutional Neural Networks for Visual Recognition](http://cs231n.github.io/convolutional-networks/#pool)

### Свёртки для текстов

Для текстов свертки работают как n-граммные детекторы (примерно). Каноничный пример символьной сверточной сети:

![text-convs](https://image.ibb.co/bC3Xun/2018_03_27_01_24_39.png =x500)  
From [Character-Aware Neural Language Models](https://arxiv.org/abs/1508.06615)

*Сколько учится фильтров на данном примере?*

На картинке показано, как из слова извлекаются 2, 3 и 4-граммы. Например, желтые - это триграммы. Желтый фильтр прикладывают ко всем триграммам в слове, а потом с помощью global max-pooling извлекают наиболее сильный сигнал.

Что это значит, если конкретнее?

Каждый символ отображается с помощью эмбеддингов в некоторый вектор. А их последовательности - в конкатенации эмбеддингов.  
Например, "abs" $\to [v_a; v_b; v_s] \in \mathbb{R}^{3 d}$, где $d$ - размерность эмбеддинга. Желтый фильтр $f_k$ имеет такую же размерность $3d$.  
Его прикладывание - это скалярное произведение $\left([v_a; v_b; v_s] \odot f_k \right) \in \mathbb R$ (один из желтых квадратиков в feature map для данного фильтра).

Max-pooling выбирает $max_i \left( [v_{i-1}; v_{i}; v_{i+1}] \odot f_k \right)$, где $i$ пробегается по всем индексам слова от 1 до $|w| - 1$ (либо по большему диапазону, если есть padding'и).   
Этот максимум соответствует той триграмме, которая наиболее близка к фильтру по косинусному расстоянию.

В результате в векторе после max-pooling'а закодирована информация о том, какие из n-грамм встретились в слове: если встретилась близкая к нашему $f_k$ триграмма, то в $k$-той позиции вектора будет стоять большое значение, иначе - маленькое.

А учим мы как раз фильтры. То есть сеть должна научиться определять, какие из n-грамм значимы, а какие - нет.

### Игрушечный пример

Посмотрим на примере, что там происходит. Возьмем слово:

In [0]:
word = 'Смирнов'

Для начала нужно перенумеровать символы:

In [0]:
char2index = {symb: ind for ind, symb in enumerate(set(word))}

char2index

Каждому символу сопоставляется эмбеддинг. Простейший способ сделать эмбеддинги - взять единичную матрицу. Когда у нас были десятки тысяч слов, такие эмбеддинги были не оч, а сейчас всего несколько символов вполне адекватно присвоить им ортогональные вектора небольшой размерности.

In [0]:
embeddings = torch.eye(len(char2index))

embeddings

Построим тензор индексов символов слова:

In [0]:
word_tensor = torch.LongTensor([char2index[symb] for symb in word])

word_tensor

Отобразим его в эмбеддинги. Получили такой же прямоугольничек, как на картинке (транспонирование нужно, чтобы смотрело в ту же сторону).

In [0]:
word_embs = embeddings[word_tensor].t()

word_embs

Теперь дело дошло до сверток. Сделаем фильтр-детектор триграммы `нов`:

In [0]:
kernel_name = 'нов'

kernel_indices = torch.LongTensor([char2index[symb] for symb in kernel_name])
kernel_weights = embeddings[kernel_indices].t()

kernel_weights

Чтобы посчитать свёртку, воспользуемся функцией:

```python
F.conv2d(input, weight, bias=None, stride=1, padding=0, dilation=1, groups=1) -> Tensor
```

input: input tensor of shape ($N \times C_{in} \times H_{in} \times W_{in}$)  
weight: filters of shape ($C_{out} \times C_{in} \times H_{out} \times W_{out}$)

$N$ - размер батча (1 у нас). $C_{in}$ - число каналов. В нашем случае оно всегда будет 1 (пока что). $C_{out}$ - число фильтров. Оно пока 1.

Нам понадобятся четырехмерные тензоры, для этого воспользуемся `view`:

In [0]:
word_embs = word_embs.view(1, 1, word_embs.shape[0], word_embs.shape[1])
kernel_weights = kernel_weights.view(1, 1, kernel_weights.shape[0], kernel_weights.shape[1])

conv_result = F.conv2d(word_embs, kernel_weights)[0, 0]

print('Conv =', conv_result)
print('Max pooling =', conv_result.max())

Свертка сказала, что данный фильтр есть на последней позиции. Пулинг сказал, пофиг на какой позиции - главное, он есть.

### Подготовка данных

Первый шаг - определить, какой длины слова у нас. Ограничимся каким-то числом, а более длинные будем обрезать.

In [0]:
from collections import Counter 
    
def find_max_len(counter, threshold):
    sum_count = sum(counter.values())
    cum_count = 0
    for i in range(max(counter)):
        cum_count += counter[i]
        if cum_count > sum_count * threshold:
            return i
    return max(counter)

word_len_counter = Counter()
for word in train_data:
    word_len_counter[len(word)] += 1
    
threshold = 0.99
MAX_WORD_LEN = find_max_len(word_len_counter, threshold)

print('Max word length for {:.0%} of words is {}'.format(threshold, MAX_WORD_LEN))

Соберем отображение из символов в индексы.

In [0]:
chars = set()
for word in train_data:
    chars.update(word)

char_index = {c : i + 1 for i, c in enumerate(chars)}
char_index['<pad>'] = 0
    
print(char_index)

**Задание** Сконвертируйте данные

In [0]:
def convert_data(data, max_word_len, char_index):
    return <np array>

X_train = convert_data(train_data, MAX_WORD_LEN, char_index)
X_test = convert_data(test_data, MAX_WORD_LEN, char_index)

In [0]:
def iterate_batches(X, y, batch_size):
    num_samples = X.shape[0]

    indices = np.arange(num_samples)
    np.random.shuffle(indices)
    
    for start in range(0, num_samples, batch_size):
        end = min(start + batch_size, num_samples)
        
        batch_idx = indices[start: end]
        
        yield X[batch_idx], y[batch_idx]

### MyFirstConvCharNN

Теперь построим свёрточную модель.

Пусть она будет строить триграммы - то есть применять фильтры на 3 символа.

Начнем с последовательности: `nn.Embedding -> nn.Conv2d -> nn.ReLU -> max pooling -> nn.Linear`

`nn.Conv2d` - это слой, содержащий создание и инициализацию фильтров, и вызов `F.conv2d` к ним и входу.

*Лайфхак:* последовательности операций можно запаковывать в `nn.Sequential`.

In [0]:
class ConvClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim, filters_count):
        super().__init__()
        
        self._embedding = ...
        self._dropout = nn.Dropout(0.2)
        self._conv3 = ...
        self._out_layer = ...
        
    def forward(self, inputs):
        '''
        inputs - LongTensor with shape (batch_size, max_word_len)
        outputs - FloatTensor with shape (batch_size,)
        '''
        
        outputs = self.embed(inputs)
        return self._out_layer(outputs).squeeze(-1)
    
    def embed(self, inputs):
        <calc word embedding>

Проверьте, что всё работает:

In [0]:
X_batch, y_batch = next(iterate_batches(X_train, train_labels, 32))
X_batch, y_batch = torch.LongTensor(X_batch), torch.LongTensor(y_batch)

model = ConvClassifier(len(char_index) + 1, 24, 64)
logits = model(X_batch)

**Задание** Подсчитайте precision, recall и F1-score для полученных предсказаний.

In [0]:
<calc precision, recall, f1-score>

**Задание** Напишем теперь цикл обучения, который не слишком сложно будет переиспользовать

In [0]:
import math
import time

def do_epoch(model, criterion, data, batch_size, optimizer=None):
    epoch_loss, epoch_tp, epoch_fp, epoch_fn = 0, 0, 0, 0
    
    is_train = not optimizer is None
    model.train(is_train)
    
    data, labels = data
    batchs_count = math.ceil(data.shape[0] / batch_size)
    
    with torch.autograd.set_grad_enabled(is_train):
        for i, (X_batch, y_batch) in enumerate(iterate_batches(data, labels, batch_size)):
            X_batch, y_batch = LongTensor(X_batch), FloatTensor(y_batch)

            logits = <calc logits>

            loss = <calc loss>
            epoch_loss += loss.item()

            if is_train:
                <how to optimize the beast?>

            <u can move the stuff to some function>
            tp = <calc true positives>
            fp = <calc false positives>
            fn = <calc false negatives>

            precision = ...
            recall = ...
            f1 = ...
            
            epoch_tp += tp
            epoch_fp += fp
            epoch_fn += fn

            print('\r[{} / {}]: Loss = {:.4f}, Precision = {:.2%}, Recall = {:.2%}, F1 = {:.2%}'.format(
                  i, batchs_count, loss.item(), precision, recall, f1), end='')
        
    precision = ...
    recall = ...
    f1 = ...
        
    return epoch_loss / batchs_count, recall, precision, f1

def fit(model, criterion, optimizer, train_data, epochs_count=1, 
        batch_size=32, val_data=None, val_batch_size=None):
    if not val_data is None and val_batch_size is None:
        val_batch_size = batch_size
        
    for epoch in range(epochs_count):
        start_time = time.time()
        train_loss, train_recall, train_precision, train_f1 = do_epoch(
            model, criterion, train_data, batch_size, optimizer
        )
        
        output_info = '\rEpoch {} / {}, Epoch Time = {:.2f}s: Train Loss = {:.4f}, Precision = {:.2%}, Recall = {:.2%}, F1 = {:.2%}'
        if not val_data is None:
            val_loss, val_recall, val_precision, val_f1 = do_epoch(model, criterion, val_data, val_batch_size, None)
            
            epoch_time = time.time() - start_time
            output_info += ', Val Loss = {:.4f}, Precision = {:.2%}, Recall = {:.2%}, F1 = {:.2%}'
            print(output_info.format(epoch+1, epochs_count, epoch_time, 
                                     train_loss, train_recall, train_precision, train_f1,
                                     val_loss, val_recall, val_precision, val_f1))
        else:
            epoch_time = time.time() - start_time
            print(output_info.format(epoch+1, epochs_count, epoch_time, train_loss))

In [0]:
model = ConvClassifier(len(char_index) + 1, 24, 128).cuda()

criterion = nn.BCEWithLogitsLoss().cuda()

optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad])

fit(model, criterion, optimizer, train_data=(X_train, train_labels), epochs_count=200, 
    batch_size=512, val_data=(X_test, test_labels), val_batch_size=1024)

**Задание** Проверьте работу классификатора на вашей фамилии.

Нужно не забыть перевести модель в режим инференса - некоторые слои на трейне и инференсе ведут себя по-разному.

In [0]:
model.eval()

surname = "..."
surname_tensor = ...
print('P({} is surname) = {}'.format(surname, torch.sigmoid(model(surname_tensor))))

**Задание** Постройте precision-recall curve для данного классификатора и предыдущей модели

## Визуализации

### Визуализация эмбеддингов

**Задание** Визуализируем эмбеддинги слов, как это делали раньше

In [0]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook

from sklearn.manifold import TSNE
from sklearn.preprocessing import scale


def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    output_notebook()
    
    if isinstance(color, str): 
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: 
        pl.show(fig)
    return fig


def get_tsne_projection(word_vectors):
    tsne = TSNE(n_components=2, verbose=100)
    return scale(tsne.fit_transform(word_vectors))
    
    
def visualize_embeddings(embeddings, token, colors):
    tsne = get_tsne_projection(embeddings)
    draw_vectors(tsne[:, 0], tsne[:, 1], color=colors, token=token)

In [0]:
word_indices = np.random.choice(np.arange(len(test_data)), 1000, replace=False)
words = [test_data[ind] for ind in word_indices]
labels = test_labels[word_indices]

word_tensor = convert_data(words, max(len(x) for x in words), char_index)
embeddings = <calc embeddings>

colors = ['red' if label else 'blue' for label in labels]

visualize_embeddings(embeddings, words, colors)

### Визуализация полученных свёрток

Кроме всего прочего у нас тут логистическая регрессия сверху. Можно визуализировать ее также, как в eli5.

**Задание** Добиться этого.

In [0]:
model.eval()

word = 'Смирнов'

Посчитайте вероятность, что слово - фамилия.

In [0]:
inputs = word -> LongTensor
prob = torch.sigmoid(model(inputs)).item()

Посчитайте результат свертки и пулинга

In [0]:
convs = ...
maxs, positions = convs.squeeze().max(-1)

Домножьте выход пулинга на веса выходного слоя

In [0]:
linear_weights = ...

Посчитайте веса символов: каждый фильтр прикладывается к какой-то позиции - прибавим его вес к накрываемым символам.

In [0]:
symb_weights = ...

Визуализируем это:

In [0]:
from IPython.core.display import HTML

def get_color_hex(weight):
    cmap = plt.get_cmap("RdYlGn")
    rgba = cmap(weight, bytes=True)
    return '#%02X%02X%02X' % rgba[:3]

symb_template = '<span style="background-color: {color_hex}">{symb}</span>'
res = '<p>P(surname) = {:.2%}</p>'.format(prob)
for symb, weight in zip(word, symb_weights):
    res += symb_template.format(color_hex=get_color_hex(weight), symb=symb)
res = '<p>' + res + '</p>'

HTML(res)

Объединим все в функции:

In [0]:
def calc_weights(word):
    <calc>
    
    return prob, symb_weights

def visualize(word):
    prob, symb_weights = calc_weights(word)
    
    symb_template = '<span style="background-color: {color_hex}">{symb}</span>'
    res = '<p>P(surname) = {:.2%}</p>'.format(prob)
    for symb, weight in zip(word, symb_weights):
        res += symb_template.format(color_hex=get_color_hex(weight), symb=symb)
    res = '<p>' + res + '</p>'
    return HTML(res)


visualize('Королев')

## Улучшение модели

**Задание** Для улучшение стабильности модели стоит добавить дропаут `nn.Dropout` - способ занулять часть весов на каждой эпохе для регуляризации модели. Попробуйте добавить его после эмбеддингов и после свертки (а можно еще где-нибудь).

![](https://cdn-images-1.medium.com/max/1044/1*iWQzxhVlvadk6VAJjsgXgg.png =x300)


**Задание** Другой способ регуляризовывать модель - использовать BatchNormalization (`nn.BatchNorm2d`). Попробуйте добавить его после свертки.

**Задание** Еще способ улучшить модель - добавить сверток. Реализуйте модель как на картинке в начале ноутбука: со свертками на 2, 3, 4 символа.

**Задание** Различают Narrow и Wide свёртки - по сути, добавляется ли нулевой паддинг или нет. Для текстов эта разница выглядит так:  
![narrow_vs_wide](https://image.ibb.co/eqGZaS/2018_03_28_11_23_17.png)
*From Neural Network Methods in Natural Language Processing.*

Слева - паддинг отсутствует, справа - есть. Попробуйте добавить паддинг и посмотреть, что получится. Потенциально он поможет выучить хорошие префиксы слова.

# Дополнительные материалы

## Почитать

### Основы
[Convolutional Neural Networks, cs231n](http://cs231n.github.io/convolutional-networks/)  
[Understanding Convolutions, Christopher Olah](http://colah.github.io/posts/2014-07-Understanding-Convolutions/)  
[Understanding Convolutional Neural Networks for NLP, Denny Britz](http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)

### Статьи
[Character-Aware Neural Language Models, Yoon Kim et al, 2015](https://arxiv.org/abs/1508.06615)  
[Character-level Convolutional Networks for Text Classification, Zhang et al., 2015](https://arxiv.org/abs/1509.01626)  
[A Sensitivity Analysis of (and Practitioners' Guide to) Convolutional Neural Networks for Sentence Classification Zhang et al., 2015](https://arxiv.org/abs/1510.03820)
[Learning Character-level Representations for Part-of-Speech Tagging, dos Santos et al, 2014](http://proceedings.mlr.press/v32/santos14.pdf)

## Посмотреть
[cs224n "Lecture 13: Convolutional Neural Networks"](https://www.youtube.com/watch?v=Lg6MZw_OOLI)

# Сдача задания

[Форма](https://goo.gl/forms/FfMnyNGI2P4xo0QD3)